# Автодополнение текста: LSTM и Transformer

Этот ноутбук запускает полный пайплайн проекта:

1. Подготовка датасета (train / val / test) из `data/raw_dataset.txt`.
2. Обучение LSTM-модели автодополнения и замер метрики ROUGE.
3. Оценка трансформера `distilgpt2` из `Transformers` и замер метрики ROUGE + вывод примеров предсказаний.

Просто выполните следующую ячейку, чтобы последовательно запустить все шаги.


In [22]:
import os
import sys
from pathlib import Path

# Добавляем папку src в sys.path, чтобы можно было импортировать модули проекта
sys.path.append(os.path.join(os.getcwd(), "src"))

from data_utils import load_and_preprocess_text, train_val_test_split

DATA_DIR = Path("./data")
RAW_PATH = DATA_DIR / "raw_dataset.txt"

print("1/3: Подготовка датасета из raw_dataset.txt...")

if not RAW_PATH.exists():
    raise FileNotFoundError(
        f"Не найден файл с сырыми данными: {RAW_PATH}. Поместите raw_dataset.txt в папку data."
    )

splittedtext = load_and_preprocess_text(str(RAW_PATH))
train_tokens, val_tokens, test_tokens = train_val_test_split(splittedtext)

DATA_DIR.mkdir(parents=True, exist_ok=True)

with (DATA_DIR / "train_tokens.txt").open("w", encoding="utf-8") as f_train:
    for sent in train_tokens:
        f_train.write(" ".join(sent) + "\n")

with (DATA_DIR / "val_tokens.txt").open("w", encoding="utf-8") as f_val:
    for sent in val_tokens:
        f_val.write(" ".join(sent) + "\n")

with (DATA_DIR / "test_tokens.txt").open("w", encoding="utf-8") as f_test:
    for sent in test_tokens:
        f_test.write(" ".join(sent) + "\n")

print(
    f"Готово. Кол-во предложений: train={len(train_tokens)}, val={len(val_tokens)}, test={len(test_tokens)}"
)


1/3: Подготовка датасета из raw_dataset.txt...
["- awww, that's a bummer. you shoulda got david carr of third day to do it.", "is upset that he can't update his facebook by texting it... and might cry as a result school today also. blah!", 'i dived many times for the ball. managed to save 50% the rest go out of bounds', 'my whole body feels itchy and like its on fire', "no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there."]
Готово. Кол-во предложений: train=40168, val=5021, test=5021


In [23]:
import os
import sys

# Убеждаемся, что папка src есть в sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

from lstm_train import train_lstm_model

print("2/3: Обучение LSTM-модели и замер ROUGE на валидации...")
train_lstm_model()
print("Обучение LSTM завершено, модель сохранена в models/lstm_autocomplete.pt")



2/3: Обучение LSTM-модели и замер ROUGE на валидации...
Используемое устройство: cuda
Размер словаря: 60555
Эпоха 1/5 | train loss: 7.9380 | ROUGE-1: 0.0267 | ROUGE-2: 0.0001 | ROUGE-L: 0.0267 | Test ROUGE-1: 0.0259 | Test ROUGE-2: 0.0000 | Test ROUGE-L: 0.0258
Эпоха 2/5 | train loss: 7.3931 | ROUGE-1: 0.0372 | ROUGE-2: 0.0001 | ROUGE-L: 0.0365 | Test ROUGE-1: 0.0394 | Test ROUGE-2: 0.0000 | Test ROUGE-L: 0.0385
Эпоха 3/5 | train loss: 7.2789 | ROUGE-1: 0.0634 | ROUGE-2: 0.0031 | ROUGE-L: 0.0632 | Test ROUGE-1: 0.0655 | Test ROUGE-2: 0.0036 | Test ROUGE-L: 0.0653
Эпоха 4/5 | train loss: 7.1402 | ROUGE-1: 0.0598 | ROUGE-2: 0.0032 | ROUGE-L: 0.0595 | Test ROUGE-1: 0.0596 | Test ROUGE-2: 0.0028 | Test ROUGE-L: 0.0588
Эпоха 5/5 | train loss: 7.0065 | ROUGE-1: 0.0600 | ROUGE-2: 0.0032 | ROUGE-L: 0.0596 | Test ROUGE-1: 0.0590 | Test ROUGE-2: 0.0026 | Test ROUGE-L: 0.0584

Примеры предсказаний LSTM на валидационной выборке:
=== Пример 1 ===
Префикс (3/4): i called you
Истинное продолжение (1/

In [24]:
import os
import sys

# Убеждаемся, что папка src есть в sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

from eval_transformer_pipeline import evaluate_distilgpt2_on_val

print("3/3: Оценка трансформера distilgpt2 на валидационной выборке...")
evaluate_distilgpt2_on_val()
print("Оценка distilgpt2 завершена.")



3/3: Оценка трансформера distilgpt2 на валидационной выборке...


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are no

Оценка модели distilgpt2 на валидационной выборке (181 примеров):
ROUGE-1: 0.0754 | ROUGE-2: 0.0124 | ROUGE-L: 0.0749

Примеры предсказаний:

=== Пример 1 ===
Префикс (3/4): i called you
Истинное продолжение (1/4): twice
Предсказанное продолжение:   a

=== Пример 2 ===
Префикс (3/4): i find that, in this age of internettery, this is how you perfectly express sorrow
Истинное продолжение (1/4): and empathy for another's pain
Предсказанное продолжение:   .”

=== Пример 3 ===
Префикс (3/4): sore head. go
Истинное продолжение (1/4): away
Предсказанное продолжение:   to

=== Пример 4 ===
Префикс (3/4): going to bed. i wish the hubby had
Истинное продолжение (1/4): more time off
Предсказанное продолжение:   been able to

=== Пример 5 ===
Префикс (3/4): has to study
Истинное продолжение (1/4): hard today
Предсказанное продолжение:   the effects

Оценка distilgpt2 завершена.
